# CITS 工作流程測試筆記本
# CITS Workflow Test Notebook

此筆記本示範使用 KEEN 函式庫進行 CITS 數據分析的完整工作流程。  
This notebook demonstrates the complete CITS data analysis workflow using KEEN library functions.

**主要功能 / Main Features:**
1. CITS 偏壓切片顯示（可調整索引）/ CITS bias slice display (adjustable index)
2. 線剖面能帶圖 / Line profile band diagram
3. 堆疊光譜圖（可調整偏移）/ Stacked spectra plot (adjustable offset)

**設計理念 / Design Philosophy:**
- 使用函式庫函數，最少自定義程式碼 / Use library functions with minimal custom code
- 直接調用 visualization 模組的繪圖函數 / Direct calls to visualization module plotting functions
- 清晰的數據流：載入 → 提取 → 繪圖 / Clear data flow: load → extract → plot

## 設定區域 / Configuration Section

**請在此修改您的檔案路徑和參數 / Please modify your file paths and parameters here**

In [1]:
# ============== 設定區域 / Configuration Section ==============

# 檔案路徑設定 / File path configuration
TXT_FILE_PATH = "/Users/yangziliang/Git-Projects/keen/testfile/20250521_Janus Stacking SiO2_13K_113.txt"

# CITS 檔案名稱（無副檔名）/ CITS file name (without extension)
# 注意：如果指定的檔案不存在，程式會自動選擇第一個可用的 CITS 檔案
# Note: If the specified file doesn't exist, the program will automatically select the first available CITS file
CITS_FILE_KEY = "Lia1R_Matrix"  # 常見選項: *_Matrix (例如 Lia1R_Matrix, Lia1Y_Matrix, Lia2R_Matrix)

# 分析參數 / Analysis parameters
BIAS_INDEX = 50          # 偏壓切片索引 / Bias slice index (0 to n_bias-1)
LINE_START = (10, 10)    # 線剖面起點 / Line profile start point (x, y)
LINE_END = (50, 50)      # 線剖面終點 / Line profile end point (x, y)
OFFSET_FACTOR = 1.0      # 堆疊光譜偏移係數 / Stacked spectra offset factor

# 顯示參數 / Display parameters  
USE_LOG_SCALE = False    # 能帶圖是否使用對數尺度 / Whether to use log scale for band diagram
MAX_CURVES = 15          # 堆疊光譜最大顯示曲線數 / Maximum curves for stacked spectra

# ================================================================

print("⚙️ 設定完成 / Configuration completed")
print(f"📁 TXT 檔案: {TXT_FILE_PATH}")
print(f"🔍 CITS 檔案: {CITS_FILE_KEY} (將自動檢查是否存在)")

⚙️ 設定完成 / Configuration completed
📁 TXT 檔案: /Users/yangziliang/Git-Projects/keen/testfile/20250521_Janus Stacking SiO2_13K_113.txt
🔍 CITS 檔案: Lia1R_Matrix (將自動檢查是否存在)


## 1. 初始化和數據載入 / Initialization and Data Loading

In [2]:
# 導入必要的模組 / Import necessary modules
import sys
import os
import numpy as np
sys.path.append(os.path.join(os.getcwd(), '..'))

from core.experiment_session import ExperimentSession
from core.visualization.spectroscopy_plots import SpectroscopyPlotting
from core.analysis.cits_analysis import (
    extract_cits_bias_slice,
    extract_line_spectra_data,
    extract_point_spectrum,
    prepare_stacked_spectra_data
)

print("✅ 模組導入完成 / Modules imported successfully")

✅ 模組導入完成 / Modules imported successfully


In [3]:
# 載入實驗數據 / Load experiment data
print("📂 載入檔案...")
try:
    session = ExperimentSession(TXT_FILE_PATH)
    print(f"✅ 實驗: {session.experiment_name}")
    
    available = session.available_files
    total_files = sum(len(files) for files in available.values())
    print(f"📁 總檔案數: {total_files}")
    
    # 列出可用的檔案 / List available files
    print("\n📋 可用檔案列表:")
    for file_type, file_list in available.items():
        if file_list:
            print(f"  {file_type.upper()}:")
            for key in file_list:
                print(f"    - {key}")
        
except Exception as e:
    print(f"❌ 載入失敗: {e}")
    raise

📂 載入檔案...
✅ 實驗: Unknown
📁 總檔案數: 17

📋 可用檔案列表:
  TXT:
    - 20250521_Janus Stacking SiO2_13K_113
  TOPO:
    - 20250521_Janus Stacking SiO2_13K_113TopoFwd
    - 20250521_Janus Stacking SiO2_13K_113TopoBwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1XFwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1XBwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1YFwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1YBwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1RFwd
    - 20250521_Janus Stacking SiO2_13K_113Lia1RBwd
    - 20250521_Janus Stacking SiO2_13K_113Lia2RFwd
    - 20250521_Janus Stacking SiO2_13K_113Lia2RBwd
    - 20250521_Janus Stacking SiO2_13K_113It_to_PCFwd
    - 20250521_Janus Stacking SiO2_13K_113It_to_PCBwd
  CITS:
    - 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix
    - 20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix
    - 20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix
    - 20250521_Janus Stacking SiO2_13K_113Lia2R_Matrix


In [4]:
# 載入 CITS 數據 / Load CITS data
print(f"🔍 檢查可用的 CITS 檔案...")

# 先檢查可用的 CITS 檔案
available_cits = session.available_files['cits']
print(f"📋 可用的 CITS 檔案: {available_cits}")

# 如果設定的檔案不存在，使用第一個可用的 CITS 檔案
if CITS_FILE_KEY not in available_cits:
    if available_cits:
        CITS_FILE_KEY = available_cits[0]
        print(f"⚠️  原設定檔案不存在，改用: {CITS_FILE_KEY}")
    else:
        raise ValueError("❌ 沒有找到任何 CITS 檔案")

print(f"🔍 載入 CITS 檔案: {CITS_FILE_KEY}")
try:
    cits_file = session[CITS_FILE_KEY]
    cits_data = cits_file.data
    
    print(f"✅ CITS 數據載入成功")
    print(f"📊 數據形狀: {cits_data.shape}")
    print(f"⚡ 偏壓範圍: {cits_data.bias_range[0]:.3f}V 到 {cits_data.bias_range[1]:.3f}V")
    print(f"🔢 偏壓點數: {cits_data.n_bias_points}")
    print(f"📐 網格大小: {cits_data.grid_size}")
    
    # 準備用於分析函數的數據格式 / Prepare data format for analysis functions
    cits_data_dict = {
        'data_3d': cits_data.data_3d,
        'bias_values': cits_data.bias_values,
        'grid_size': cits_data.grid_size,
        'measurement_mode': 'CITS',
        'x_range': cits_data.x_range,
        'y_range': cits_data.y_range
    }
    
except Exception as e:
    print(f"❌ CITS 數據載入失敗: {e}")
    raise

🔍 檢查可用的 CITS 檔案...
📋 可用的 CITS 檔案: ['20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia2R_Matrix']
⚠️  原設定檔案不存在，改用: 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix
🔍 載入 CITS 檔案: 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix
✅ CITS 數據載入成功
📊 數據形狀: (401, 100, 100)
⚡ 偏壓範圍: -2050.000V 到 1050.000V
🔢 偏壓點數: 401
📐 網格大小: [100, 100]


/Users/yangziliang/Git-Projects/keen/backend/test/../core/parsers/dat_parser.py:84: DtypeWarning:

Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.



## 2. CITS 偏壓切片顯示 / CITS Bias Slice Display

使用函式庫函數顯示特定偏壓的 CITS 切片  
Using library functions to display CITS slice at specific bias

In [5]:
# 提取偏壓切片數據 / Extract bias slice data
print(f"🔪 提取偏壓切片，索引: {BIAS_INDEX}")
try:
    bias_slice_info = extract_cits_bias_slice(cits_data_dict, BIAS_INDEX)
    
    print(f"✅ 切片提取成功")
    print(f"⚡ 偏壓值: {bias_slice_info['bias_value']:.3f}V")
    print(f"📊 切片形狀: {bias_slice_info['slice_data'].shape}")
    
except Exception as e:
    print(f"❌ 偏壓切片提取失敗: {e}")
    raise

🔪 提取偏壓切片，索引: 50
✅ 切片提取成功
⚡ 偏壓值: -1275.000V
📊 切片形狀: (100, 100)


In [6]:
# 繪製偏壓切片 / Plot bias slice
print("🎨 繪製 CITS 偏壓切片...")
try:
    fig_bias_slice = SpectroscopyPlotting.plot_cits_bias_slice(
        data_3d=cits_data_dict['data_3d'],
        bias_values=cits_data_dict['bias_values'],
        bias_index=BIAS_INDEX,
        title=f"CITS 偏壓切片 @ {bias_slice_info['bias_value']:.3f}V",
        colorscale='Viridis',
        width=700,
        height=700
    )
    
    fig_bias_slice.show()
    print("✅ 偏壓切片繪製完成")
    
except Exception as e:
    print(f"❌ 偏壓切片繪製失敗: {e}")
    raise

🎨 繪製 CITS 偏壓切片...


✅ 偏壓切片繪製完成


## 3. 線剖面能帶圖 / Line Profile Band Diagram

提取線剖面光譜數據並繪製能帶圖  
Extract line profile spectra data and plot band diagram

In [7]:
# 提取線剖面光譜數據 / Extract line profile spectra data
print(f"📏 提取線剖面: {LINE_START} → {LINE_END}")
try:
    line_data = extract_line_spectra_data(
        cits_data_dict, 
        LINE_START, 
        LINE_END, 
        sampling_method='bresenham'
    )
    
    print(f"✅ 線剖面提取成功")
    print(f"📊 光譜數據形狀: {line_data['line_spectra'].shape}")
    print(f"📐 物理長度: {line_data['physical_length']:.2f}")
    print(f"🔢 採樣點數: {line_data['n_points']}")
    
except Exception as e:
    print(f"❌ 線剖面提取失敗: {e}")
    raise

📏 提取線剖面: (10, 10) → (50, 50)
✅ 線剖面提取成功
📊 光譜數據形狀: (201, 41)
📐 物理長度: 56.57
🔢 採樣點數: 41


In [8]:
# 繪製能帶圖 / Plot band diagram
print("🎨 繪製能帶圖...")
try:
    fig_band = SpectroscopyPlotting.plot_band_diagram(
        line_spectra=line_data['line_spectra'],
        bias_values=line_data['bias_values'],
        distances=line_data['distances'],
        title=f"能帶圖 - 線剖面 {LINE_START}→{LINE_END}",
        use_log_scale=USE_LOG_SCALE,
        colorscale='Viridis',
        width=900,
        height=600
    )
    
    fig_band.show()
    print("✅ 能帶圖繪製完成")
    
except Exception as e:
    print(f"❌ 能帶圖繪製失敗: {e}")
    raise

🎨 繪製能帶圖...


✅ 能帶圖繪製完成


## 4. 堆疊光譜圖 / Stacked Spectra Plot

顯示多條 STS 光譜曲線，帶垂直偏移  
Display multiple STS spectra curves with vertical offset

In [9]:
# 準備堆疊光譜數據 / Prepare stacked spectra data
print(f"📚 準備堆疊光譜數據，最大曲線數: {MAX_CURVES}")
try:
    stacked_data = prepare_stacked_spectra_data(
        line_spectra=line_data['line_spectra'],
        bias_values=line_data['bias_values'],
        max_curves=MAX_CURVES,
        step_selection='uniform'
    )
    
    print(f"✅ 堆疊數據準備完成")
    print(f"📊 選定曲線數: {stacked_data['n_selected']} / {stacked_data['total_positions']}")
    print(f"🔢 選定位置: {stacked_data['selected_indices'][:5]}...")
    
except Exception as e:
    print(f"❌ 堆疊數據準備失敗: {e}")
    raise

📚 準備堆疊光譜數據，最大曲線數: 15
✅ 堆疊數據準備完成
📊 選定曲線數: 21 / 41
🔢 選定位置: [0, 2, 4, 6, 8]...


In [10]:
# 繪製堆疊光譜圖 / Plot stacked spectra
print("🎨 繪製堆疊光譜圖...")
try:
    fig_stacked = SpectroscopyPlotting.plot_stacked_spectra(
        line_spectra=stacked_data['selected_spectra'],
        bias_values=stacked_data['bias_values'],
        offset_factor=OFFSET_FACTOR,
        positions=stacked_data['selected_positions'],
        max_curves=MAX_CURVES,
        title=f"堆疊光譜圖 - 線剖面 {LINE_START}→{LINE_END}",
        width=900,
        height=700
    )
    
    fig_stacked.show()
    print("✅ 堆疊光譜圖繪製完成")
    
except Exception as e:
    print(f"❌ 堆疊光譜圖繪製失敗: {e}")
    raise

🎨 繪製堆疊光譜圖...


✅ 堆疊光譜圖繪製完成


## 5. 額外示範：單點光譜提取 / Additional Demo: Single Point Spectrum

示範如何提取和繪製單點 STS 光譜  
Demonstrate how to extract and plot single point STS spectrum

In [11]:
# 提取單點光譜 / Extract single point spectrum
POINT_X, POINT_Y = 30, 30  # 設定要分析的點座標 / Set point coordinates to analyze

print(f"📍 提取單點光譜，座標: ({POINT_X}, {POINT_Y})")
try:
    point_data = extract_point_spectrum(cits_data_dict, POINT_X, POINT_Y)
    
    print(f"✅ 單點光譜提取成功")
    print(f"📊 光譜點數: {point_data['n_points']}")
    print(f"⚡ 偏壓範圍: {point_data['bias_values'][0]:.3f}V 到 {point_data['bias_values'][-1]:.3f}V")
    
except Exception as e:
    print(f"❌ 單點光譜提取失敗: {e}")
    raise

📍 提取單點光譜，座標: (30, 30)
✅ 單點光譜提取成功
📊 光譜點數: 401
⚡ 偏壓範圍: -2050.000V 到 -2050.000V


/Users/yangziliang/Git-Projects/keen/backend/test/../core/analysis/cits_analysis.py:891: RuntimeWarning:

divide by zero encountered in scalar divide



In [12]:
# 繪製單點 STS 光譜 / Plot single point STS spectrum
print("🎨 繪製單點 STS 光譜...")
try:
    fig_point = SpectroscopyPlotting.plot_sts_spectrum(
        bias_values=point_data['bias_values'],
        current=point_data['current'],
        conductance=point_data['conductance'],
        title=f"STS 光譜 @ 座標 ({POINT_X}, {POINT_Y})",
        show_conductance=True,
        width=800,
        height=500
    )
    
    fig_point.show()
    print("✅ 單點 STS 光譜繪製完成")
    
except Exception as e:
    print(f"❌ 單點 STS 光譜繪製失敗: {e}")
    raise

🎨 繪製單點 STS 光譜...


✅ 單點 STS 光譜繪製完成


## 總結 / Summary

**工作流程完成！/ Workflow Complete!**

✅ **已完成的分析 / Completed Analysis:**
1. CITS 偏壓切片顯示 / CITS bias slice display
2. 線剖面能帶圖 / Line profile band diagram  
3. 堆疊光譜圖 / Stacked spectra plot
4. 單點 STS 光譜 / Single point STS spectrum

**主要特點 / Key Features:**
- 🔧 **函式庫導向**: 使用標準化的分析和繪圖函數 / Library-oriented: Using standardized analysis and plotting functions
- 🎯 **最少自定義程式碼**: 專注於數據分析而非程式實作 / Minimal custom code: Focus on data analysis rather than implementation
- 📊 **完整視覺化**: 包含所有主要的 CITS 分析圖表 / Complete visualization: Including all major CITS analysis plots
- ⚙️ **參數化設計**: 易於調整和重複使用 / Parameterized design: Easy to adjust and reuse

**調整參數以測試不同條件 / Adjust parameters to test different conditions:**
- 修改 `BIAS_INDEX` 查看不同偏壓的切片 / Modify `BIAS_INDEX` to view slices at different biases
- 改變 `LINE_START` 和 `LINE_END` 分析不同的線剖面 / Change `LINE_START` and `LINE_END` to analyze different line profiles
- 調整 `OFFSET_FACTOR` 改變堆疊光譜的分離度 / Adjust `OFFSET_FACTOR` to change stacked spectra separation